In [1]:
#% pip install polars

In [2]:
import polars as pl

In [3]:
mi_serie = pl.Series(
    name="mi_serie",
    values=[1, 2, 3, 4, 5],
).cast(pl.Int32)
mi_serie

mi_serie
i32
1
2
3
4
5


In [4]:
mi_serie.filter(mi_serie > 2)

mi_serie
i32
3
4
5


In [5]:
print(f'El valor maximo es {mi_serie.max()}')

El valor maximo es 5


In [6]:
df = pl.DataFrame(
    {
        "Pokemon": ["Pikachu", "Charmander", "Squirtle", "Bulbasaur"],
        "Tipo 1": ["Electrico", "Fuego", "Agua", "Planta"],
        "Tipo 2": [None, None, None, "Veneno"],
        "HP" : [35, 39, 44, 45],
    },
    schema={
        "Pokemon": pl.Utf8,
        "Tipo 1": pl.Utf8,
        "Tipo 2": pl.Utf8,
        "HP": pl.Int32,
    }
)
df

Pokemon,Tipo 1,Tipo 2,HP
str,str,str,i32
"""Pikachu""","""Electrico""",null,35
"""Charmander""","""Fuego""",null,39
"""Squirtle""","""Agua""",null,44
"""Bulbasaur""","""Planta""","""Veneno""",45


In [7]:
# Simular que le pegaron al pokemon, le bajaron 35 HP
df = df.with_columns(
    (pl.col("HP") - 35).alias("HP")
)
df

Pokemon,Tipo 1,Tipo 2,HP
str,str,str,i32
"""Pikachu""","""Electrico""",null,0
"""Charmander""","""Fuego""",null,4
"""Squirtle""","""Agua""",null,9
"""Bulbasaur""","""Planta""","""Veneno""",10


In [8]:
# Agregar si esta vivo o muerto
df = df.with_columns(
    pl.when(
        pl.col("HP") > 0
    ).then(
        pl.lit("Vivo")
    ).otherwise(
        pl.lit("Muerto")
    ).alias(
        "Estado"
    )
)
df

Pokemon,Tipo 1,Tipo 2,HP,Estado
str,str,str,i32,str
"""Pikachu""","""Electrico""",null,0,"""Muerto"""
"""Charmander""","""Fuego""",null,4,"""Vivo"""
"""Squirtle""","""Agua""",null,9,"""Vivo"""
"""Bulbasaur""","""Planta""","""Veneno""",10,"""Vivo"""


In [9]:
# Columna para decir si es la mascota de Pokemon o no
df = df.with_columns(
    pl.when(
        "Pokemon" == "Pikachu"
    ).then(
        pl.lit(True)
    ).otherwise(
        pl.lit(False)
    ).alias("Es mascota")
)
df

Pokemon,Tipo 1,Tipo 2,HP,Estado,Es mascota
str,str,str,i32,str,bool
"""Pikachu""","""Electrico""",null,0,"""Muerto""",false
"""Charmander""","""Fuego""",null,4,"""Vivo""",false
"""Squirtle""","""Agua""",null,9,"""Vivo""",false
"""Bulbasaur""","""Planta""","""Veneno""",10,"""Vivo""",false


In [10]:
column_names = df.columns
column_names

['Pokemon', 'Tipo 1', 'Tipo 2', 'HP', 'Estado', 'Es mascota']

In [11]:
df.select(column_names[1])

Tipo 1
str
"""Electrico"""
"""Fuego"""
"""Agua"""
"""Planta"""


### Lazy frames

In [16]:
df_lazy = pl.DataFrame({
    "a": [1, 2, 3, 4, 5],
    "b": [6, 7, 8, 9, 10],
}).lazy()

df_transformed = df_lazy.filter(
    pl.col("a") > 2
).with_columns(
    (pl.col("b") + pl.col("a")).alias("suma")
)

result = df_transformed
result

In [17]:
result.collect()

a,b,suma
i64,i64,i64
3,8,11
4,9,13
5,10,15
